In [ ]:
!pip install --upgrade pip
!pip install tensorflow

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Bidirectional

In [ ]:
df = pd.read_csv('X:\RA_AirQuality\combined_final_dataset.csv', index_col='DateTime')

In [ ]:
train_split= 0.9
split_idx = int(len(df) * 0.9)
training_set = df[:split_idx].values
test_set = df[split_idx:].values

In [ ]:
# 5-day prediction using 30 days data
x_train = []
y_train = []
n_future = 5 #Next 5 days rainfall forecast
n_past = 30 #Past 30 days
for i in range(0, len(training_set) - n_past - n_future + 1):
    x_train.append(training_set[i : i + n_past, 0])
    y_train.append(training_set[i + n_past : i + n_past + n_future, 0])

In [ ]:
x_train , y_train = np.array(x_train), np.array(y_train)
x_train = np.reshape(x_train, (x_train.shape[0] , x_train.shape[1], 1))

In [ ]:
EPOCHS = 500
BATCH_SIZE = 32

regressor = Sequential()
regressor.add(Bidirectional(LSTM(units=30, return_sequences=True, input_shape = (x_train.shape[1], 1))))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units= 30, return_sequences=True))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units= 30, return_sequences=True))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units= 30))
regressor.add(Dropout(0.2))
regressor.add(Dense(units = n_future, activation=’relu’))

In [ ]:
regressor.compile(optimizer=’adam’, loss=’mean_squared_error’, metrics=[‘acc’])
regressor.fit(x_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE)